## Step 1: Restructure directory to match pytorch image dataset structure

In [1]:
import os

def restructure(dir, num_classes):
    for i in range(1, num_classes+1):
        dir_to_create = f".\\{dir}\\flower{str(i)}"
        
        if not os.path.isdir(dir_to_create):
            os.mkdir(dir_to_create)

    cwd = os.path.abspath(dir)

    for file in os.listdir(dir):
        if file.endswith(".jpg"):
            os.replace(f"{cwd}\\{file}", f"{cwd}\\flower{file.split("_")[0]}\\{file}")

restructure("train_data", 60)
restructure("val_data", 60)

## Step 2: Load dataset into torch dataloader

In [2]:
import torch.utils.data
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder("train_data", transform=transform)
val_dataset = datasets.ImageFolder("val_data", transform=transform)

batch_size = 256
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

## Step 3: Define model

In [3]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64*64*3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 60)
        )

    def forward(self, X):
        X = self.flatten(X)
        logits = self.linear_relu_stack(X)
        return logits
    
model = Model().to(device)
print(model)

Model(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=12288, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=60, bias=True)
  )
)


In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        one_hot_y = torch.eye(60)[y].to(device)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, one_hot_y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            one_hot_y = torch.eye(60)[y].to(device)
            y = y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, one_hot_y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
import torch.optim

torch.manual_seed(42)

learning_rate = 1e-3
epochs = 30

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
    test_loop(val_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4.104539  [  256/ 3000]
Test Error: 
 Accuracy: 7.2%, Avg loss: 3.893274 

Epoch 2
-------------------------------
loss: 3.921190  [  256/ 3000]
Test Error: 
 Accuracy: 13.7%, Avg loss: 3.458352 

Epoch 3
-------------------------------
loss: 3.369710  [  256/ 3000]
Test Error: 
 Accuracy: 17.7%, Avg loss: 3.124852 

Epoch 4
-------------------------------
loss: 3.112640  [  256/ 3000]
Test Error: 
 Accuracy: 18.3%, Avg loss: 3.106195 

Epoch 5
-------------------------------
loss: 2.822513  [  256/ 3000]
Test Error: 
 Accuracy: 25.2%, Avg loss: 2.865724 

Epoch 6
-------------------------------
loss: 2.824481  [  256/ 3000]
Test Error: 
 Accuracy: 25.5%, Avg loss: 2.765909 

Epoch 7
-------------------------------
loss: 2.554141  [  256/ 3000]
Test Error: 
 Accuracy: 27.7%, Avg loss: 2.783778 

Epoch 8
-------------------------------
loss: 2.344632  [  256/ 3000]
Test Error: 
 Accuracy: 28.7%, Avg loss: 2.732621 

Epoch 9
-----------------

In [7]:
idx_to_class = dict([(b, a) for a, b in train_dataset.class_to_idx.items()])
# for images, labels in train_dl:
#     print(idx_to_class[labels[0].item()])
#     imshow(images[0].permute(1, 2, 0))
#     break